Contains several code snippets for subtitle placement data generation.

# Note: This code is not supported and only provided as additional supporting material.

In [5]:
# Code snippet to rename files if they have been exported from label-studio with hash in filename
import os

path_source = "D:\Master_Thesis_results\Subtitle_Placement"

sub_folders = ["train", "test", "val"]

for folder in sub_folders:
    for filename in os.listdir(path_source + "\\" + folder):
        if os.path.isfile(os.path.join(path_source + "\\" + folder, filename)):
            new_name = filename.split("-")[-1]
            new_path = os.path.join(path_source + "\\" + folder, new_name)
            os.rename(os.path.join(path_source + "\\" + folder, filename), new_path)

In [47]:
# Code snippet to extract only relevant pixelmaps from source corpus after filtering for frames containing active speaker
import os
import shutil

filename_list_directory = r'D:\Master_Thesis_data\Subtitle_Placement\_A'
source_directory = 'E:\Python\MasterThesis\AutomaticSubtitlePlacement\data\subtitle_position_boxes_middle_of_subtitle\pixelmaps'
sub_folders = ["train", "test", "val"]

file_names_dict = {}
for folder in sub_folders:
    # source_filenames = os.listdir(filename_list_directory + "folder")
    for i in os.listdir(filename_list_directory + "\\" + folder):
        file_names_dict[i.replace("jpg", "png")] = filename_list_directory.replace("_A", "stuffthingmaps") + "\\" + folder

for root, dirs, files in os.walk(source_directory):
    for filename in files:
        source_path = os.path.join(root, filename)
        # Check if the filename exists in the file_mapping dictionary.
        if filename in file_names_dict:
            #target_path = os.path.join(target_directory, file_mapping[filename])
            target_path = file_names_dict[filename]
            # Ensure that the target directory exists before copying.
            os.makedirs(os.path.dirname(target_path + "\\" + filename), exist_ok=True)
            
            # Copy the file to the target path.
            shutil.copy(source_path, target_path)

In [1]:
# Code snippet to drop not relevant images for overlapped and voting task from the source 
# corpus (extracted with dataset_sampler) according to initial dataset filtered for active speaker

import json, os

json_pathes = [r"D:\Master_Thesis_data\Active_Speaker\data\val\result.json", 
               r"D:\Master_Thesis_data\Active_Speaker\data\test\result.json", 
               r"D:\Master_Thesis_data\Active_Speaker\data\train\result.json"]

directory_path = 'D:\Master_Thesis_data\Active_Speaker_overlapped\data'

files_list = []
deleted = 0
for path in json_pathes:
    print(path)
    with open(path, 'r') as json_file:
        data = json.load(json_file)
    
    if not files_list:
        files_list = [i["file_name"].split("-")[-1] for i in data["images"]]
        # files_list = [i["file_name"] for i in data["images"]]
    else:
        files_list.extend([i["file_name"].split("-")[-1] for i in data["images"]])
        # files_list.extend([i["file_name"] for i in data["images"]])
len(files_list)

available = 0
for root, dirs, files in os.walk(directory_path):
    for filename in files:
        file_path = os.path.join(root, filename)
        if filename not in files_list and not filename.endswith("result.json"):
            # print("i would remove file: " + file_path)
            os.remove(file_path)
            deleted += 1
        else:
            available += 1   

print(deleted)
print(available)

D:\Master_Thesis_data\Active_Speaker\data\val\result.json
D:\Master_Thesis_data\Active_Speaker\data\test\result.json
D:\Master_Thesis_data\Active_Speaker\data\train\result.json


In [7]:
# Code snippet to filter annotation files for the images remaining after filtering for active speaker, based on active speaker annotation files

import json

jsons_active_speaker = [r"D:\Master_Thesis_data\Active_Speaker\data\train\result.json", 
                        r"D:\Master_Thesis_data\Active_Speaker\data\test\result.json",
                        r"D:\Master_Thesis_data\Active_Speaker\data\val\result.json"]

jsons_subtitle_placement = [r"E:\Python\MasterThesis\AutomaticSubtitlePlacement\data\subtitle_position_boxes_middle_of_subtitle\jsons\train\results.json",
                           r"E:\Python\MasterThesis\AutomaticSubtitlePlacement\data\subtitle_position_boxes_middle_of_subtitle\jsons\test\results.json",
                           r"E:\Python\MasterThesis\AutomaticSubtitlePlacement\data\subtitle_position_boxes_middle_of_subtitle\jsons\val\results.json"]

json_output_path = r"D:\Master_Thesis_data\Subtitle_Placement\jsons"


active_speaker_jsons_dict = {}
for path in jsons_active_speaker:
    with open(path, 'r') as file:
        active_speaker_jsons_dict[path.split("\\")[-2]] = json.load(file)

subtitle_placement_jsons_dict = {}
for path in jsons_subtitle_placement:
    with open(path, 'r') as file:
        subtitle_placement_jsons_dict[path.split("\\")[-2]] = json.load(file)

id_name_mapping = {pvalue["id"]: pvalue["file_name"] for p, v in subtitle_placement_jsons_dict.items() for pvalue in v["images"] }
name_annotation_mapping = {id_name_mapping[pvalue["image_id"]]: pvalue for p, v in subtitle_placement_jsons_dict.items() for pvalue in v["annotations"]}

final_dict = {}
for pkey, pvalue in subtitle_placement_jsons_dict.items():
    final_dict[pkey] = {}
    final_dict[pkey]["info"] = pvalue["info"]
    final_dict[pkey]["licenses"] = pvalue["licenses"]
    final_dict[pkey]["categories"] = pvalue["categories"]
    final_dict[pkey]["images"] = active_speaker_jsons_dict[pkey]["images"]
    final_dict[pkey]["annotations"] = []
    for item in final_dict[pkey]["images"]:
        item["file_name"] = item["file_name"].split("-")[-1]
        final_dict[pkey]["annotations"].append(name_annotation_mapping[item["file_name"]])

for pkey, pvalue in final_dict.items():
    with open(json_output_path + r'\results_' + pkey + '.json', 'w') as file:
        json.dump(pvalue, file)

In [2]:
# # Code snippet to adjust the colour information in annotation file --> not relevant

# import os
# import cv2
# import numpy as np
# from tqdm import tqdm

# # Define a function to process images in a directory and its subdirectories
# def process_images_in_directory(root_dir, input_color, output_color):
#     for foldername, subfolders, filenames in os.walk(root_dir):
#         for filename in tqdm(filenames, desc="Processing images"):
#             file_path = os.path.join(foldername, filename)

#             # Check if the file is an image
#             if file_path.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp', '.gif')):
#                 # Load the image
#                 image = cv2.imread(file_path)

#                 # Define the input color to replace and the output color
#                 input_color = np.array(input_color)
#                 output_color = np.array(output_color)

#                 # Replace pixels with the specified color
#                 mask = np.all(image == input_color, axis=2)
#                 image[mask] = output_color

#                 # Save the modified image
#                 cv2.imwrite(file_path, image)

# # Specify the root directory and the colors to replace
# root_directory = r'D:\Gits\SAN\datasets\coco-middle_of_subtitle_empty\stuffthingmaps_detectron2'
# input_color = [200, 200, 200]
# output_color = [1, 1, 1]

# # Call the function to process the images
# process_images_in_directory(root_directory, input_color, output_color)

Processing images: 100%|███████████████████████████████████████████████████████████| 1725/1725 [01:24<00:00, 20.35it/s]
Processing images: 0it [00:00, ?it/s]
Processing images: 100%|█████████████████████████████████████████████████████████████| 345/345 [00:16<00:00, 20.49it/s]
Processing images: 0it [00:00, ?it/s]


In [ ]:
# Code snippet to drop not relevant images for overlapped and voting task from the source 
# corpus (extracted with dataset_sampler) according to initial dataset filtered for active speaker

import os
import shutil

def copy_matching_files(source_folder, source_new_folder, destination_folder):
    # Create the destination folder if it doesn't exist
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    # Get the list of files in the source folder
    source_files = set(os.listdir(source_folder))
    source_files = [i.replace(".png", ".jpg") for i in source_files]
    
    # Iterate through files in source_new_folder
    for root, dirs, files in os.walk(source_new_folder):
        for file in files:
            p_file_name = file.split("_")[0] + ".jpg"
            print(file)            
            file_path = os.path.join(root, file)

            # Check if the file exists in the source folder
            if p_file_name in source_files:
                # Copy the file to the destination folder
                shutil.copy(file_path, os.path.join(destination_folder, file))
                print(f"Copied: {file}")

# Example usage
source_folder = r"D:\Master_Thesis_data\Active_Speaker_overlapped\data\test\images"
source_new_folder = r"E:\Python\MasterThesis\AutomaticSubtitlePlacement\data\subtitle_position_boxes_middle_of_subtitle_voting\_A\test"
destination_folder = r"D:\Master_Thesis_data\Active_Speaker_voting\data\test\images"

copy_matching_files(source_folder, source_new_folder, destination_folder)